In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import pairwise_distances_argmin_min
from scipy.spatial.distance import cdist

In [4]:
data = pd.read_csv('circles.csv')

In [5]:
data.head()

,x,y,class
0,3.15676,116.12252,6
1,16.14436,16.81660,11
2,100.31212,64.99025,53
3,-1.33773,84.81772,4
4,104.37328,62.42373,53


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       10000 non-null  float64
 1   y       10000 non-null  float64
 2   class   10000 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 234.5 KB


# Scaling the data features

In [7]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[['x','y']])

# Model training

In [8]:
# assign k[class in data] as number of cluster
k = len(data['class'].unique())
kmeans = KMeans(n_clusters=k,init='k-means++',random_state=42,n_init=10,max_iter=300)
kmeans.fit(data_scaled)

KMeans(n_clusters=100, n_init=10, random_state=42)

**prediciting the data labels**

In [10]:
predicted_labels = kmeans.labels_
true_labels = data['class']


# Model evaluation

In [11]:
ari_score = adjusted_rand_score(true_labels,predicted_labels)
print(ari_score)

0.9835673071815411


# FLS++ algorithm

In [27]:
# Define FLS++ algorithm (simplified version)
def foresight_ls_kmeans(data, n_clusters, n_init=10, max_iter=300, Z=10, random_state=42):
    """
    Simplified Foresight Local Search k-means++ (FLS++) algorithm.
    
    Parameters:
    - data: ndarray, input data points
    - n_clusters: int, number of clusters
    - n_init: int, number of random initializations
    - max_iter: int, max iterations for Lloyd's algorithm
    - Z: int, number of foresight swap steps
    - random_state: int, seed for reproducibility
    """
    np.random.seed(random_state)
    best_inertia = None
    best_labels = None
    
    for _ in range(n_init):
        # Step 1: k-means++ initialization
        initial_centers = KMeans(n_clusters=n_clusters, init='k-means++', n_init=1, max_iter=1, random_state=random_state).fit(data).cluster_centers_
        
        # Step 2: Lloyd's iterations with foresight for swap trials
        centers = initial_centers.copy()
        for _ in range(Z):
            # Calculate current cost (inertia) and initial assignment
            labels, dist = pairwise_distances_argmin_min(data, centers)
            inertia = np.sum(dist ** 2)

            # Attempt to swap each center with a new point (d²-sampling-like)
            best_swap_inertia = inertia
            best_swap_centers = centers.copy()
            
            for j in range(n_clusters):
                # Randomly select a new center candidate
                candidate_index = np.random.choice(len(data))
                candidate_center = data[candidate_index].reshape(1, -1)
                
                # Swap the j-th center with the candidate
                swapped_centers = centers.copy()
                swapped_centers[j] = candidate_center
                
                # Perform one Lloyd step with swapped centers
                new_labels, new_dist = pairwise_distances_argmin_min(data, swapped_centers)
                new_inertia = np.sum(new_dist ** 2)

                # Keep the swap if it reduces the cost
                if new_inertia < best_swap_inertia:
                    best_swap_inertia = new_inertia
                    best_swap_centers = swapped_centers.copy()
            
            # Update centers after the best swap
            centers = best_swap_centers.copy()

        # Final Lloyd's iterations for convergence
        kmeans_final = KMeans(n_clusters=n_clusters, init=centers, n_init=1, max_iter=max_iter, random_state=random_state).fit(data)
        
        # Check if this initialization yields the best result
        if best_inertia is None or kmeans_final.inertia_ < best_inertia:
            best_inertia = kmeans_final.inertia_
            best_labels = kmeans_final.labels_
    
    return best_labels, best_inertia
    

**applying FLS++ on the standardized data with k as the number of unique clusters**

In [30]:
# Apply FLS++ on the standardized data with k as the number of unique classes
predicted_labels_fls, final_inertia = foresight_ls_kmeans(data_scaled, n_clusters=k, Z=10, random_state=42)

# Evaluate performance using ARI to compare with previous results
ari_score_fls = adjusted_rand_score(true_labels, predicted_labels_fls)
final_inertia, ari_score_fls

(74.91568554782283, 1.0)

In [31]:
from sklearn.utils import resample

# Step 1: Sample a subset of the data (2,000 points) to reduce computation
sampled_data, sampled_labels = resample(data_scaled, true_labels, n_samples=2000, random_state=42)

# Update the FLS++ function with a reduced number of steps and center swaps
def optimized_fls_kmeans(data, n_clusters, Z=5, random_state=42):
    """
    Optimized FLS++ with reduced foresight steps and limited center trials for swap.
    
    Parameters:
    - data: ndarray, input data points
    - n_clusters: int, number of clusters
    - Z: int, number of foresight swap steps
    - random_state: int, seed for reproducibility
    """
    np.random.seed(random_state)
    initial_centers = KMeans(n_clusters=n_clusters, init='k-means++', n_init=1, max_iter=1, random_state=random_state).fit(data).cluster_centers_
    centers = initial_centers.copy()

    for _ in range(Z):
        # Calculate initial cost and assignments
        labels, dist = pairwise_distances_argmin_min(data, centers)
        inertia = np.sum(dist ** 2)
        
        # Only swap a subset of centers for efficiency
        swap_indices = np.random.choice(range(n_clusters), size=n_clusters // 2, replace=False)
        best_swap_inertia = inertia
        best_swap_centers = centers.copy()

        for j in swap_indices:
            candidate_index = np.random.choice(len(data))
            candidate_center = data[candidate_index].reshape(1, -1)

            # Perform swap and one Lloyd step
            swapped_centers = centers.copy()
            swapped_centers[j] = candidate_center
            
            new_labels, new_dist = pairwise_distances_argmin_min(data, swapped_centers)
            new_inertia = np.sum(new_dist ** 2)

            if new_inertia < best_swap_inertia:
                best_swap_inertia = new_inertia
                best_swap_centers = swapped_centers.copy()

        centers = best_swap_centers.copy()

    # Final Lloyd's iteration for convergence
    kmeans_final = KMeans(n_clusters=n_clusters, init=centers, n_init=1, max_iter=300, random_state=random_state).fit(data)
    return kmeans_final.labels_, kmeans_final.inertia_

# Run the optimized FLS++ on the sampled data and evaluate
predicted_labels_optimized_fls, final_inertia_optimized = optimized_fls_kmeans(sampled_data, n_clusters=k, Z=5, random_state=42)
ari_score_optimized_fls = adjusted_rand_score(sampled_labels, predicted_labels_optimized_fls)
final_inertia_optimized, ari_score_optimized_fls


(17.240239990171926, 0.947713110778318)